In [1]:
#!pip install ipynb

In [2]:
import numpy as np
import threading
import random
import time
from protocol_tests import test_all
from qiskit_aer import AerSimulator
from qiskit import transpile, QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from threading import Thread, Lock
from globals import *



import sys
try:
    del sys.modules['ipynb.fs.full.weak_global_coin']
except KeyError:
    pass

import ipynb.fs.full.weak_global_coin as QCF

try:
    del sys.modules['globals']
except KeyError:
    pass

In [3]:
broadcasted_messages = []
first_to_decide = None

QUESTION_MARK = "?"
WAITING_MESSAGE = "waiting"
HALF_PLUS_ONE = int(np.floor(n/2)) + 1
MAX_ALIVE_PROCESSES = n - t

broadcasting_lock = Lock()
decision_lock = Lock()

In [4]:
class Process:
    def __init__(self, id, input_val, faulty=False):
        self.id = id
        self.input_val = input_val
        self.output = None
        self.faulty = faulty
        self.non_faulty = not faulty
        self.round_messages = {}  # holds messages received in a round
        self.decision_epoch = None

class Message:
    def __init__(self, sender_id, receivers, epoch, round, message):
        self.sender_id = sender_id
        self.receivers = receivers
        self.epoch = epoch
        self.round = round
        self.message = message
        self.read = {pid: False for pid in receivers}

In [5]:
def broadcast(process_id, epoch, round, message):
    print(f"[Process {process_id}] BROADCAST round {round}, epoch {epoch} → {message}")
    new_msg = Message(process_id, list(range(n)), epoch, round, message)
    broadcasting_lock.acquire()
    broadcasted_messages.append(new_msg)
    broadcasting_lock.release()

def waiting_condition(num_received_messages, round):
    print(f"num_received_messages: {num_received_messages}, round: {round}")    
    if round in [1, 2]:
        actual_alive = sum([1 for p in processes if p.non_faulty])
        print(f"actual_alive: ", actual_alive)
        return num_received_messages < actual_alive
    elif round == 3:
        return num_received_messages < MAX_ALIVE_PROCESSES      # all n - t honest processes must send "WAIT" in round 3

def receive(process, epoch, round, required_val=None):
    print(f"[Process {process.id}] WAITING to receive round {round} @ epoch {epoch}")
    num_received_messages = 0
    start_time = time.time()

    while waiting_condition(num_received_messages, round):
        if time.time() - start_time > 10:
            print(f"[Process {process.id}] TIMEOUT in round {round}, epoch {epoch}, only got {num_received_messages}")
            break

        broadcasting_lock.acquire()
        for msg in broadcasted_messages:
            if msg.epoch == epoch and msg.round == round and process.id in msg.receivers and not msg.read[process.id]:
                if round == 3:
                    assert msg.message == required_val
                count = process.round_messages.get(msg.message, 0)
                process.round_messages[msg.message] = count + 1
                msg.read[process.id] = True
                num_received_messages += 1
        broadcasting_lock.release()

    print(f"[Process {process.id}] RECEIVED {num_received_messages} msgs in round {round}, epoch {epoch}")


def get_majority_value(process):
    for value, count in process.round_messages.items():
        if count >= HALF_PLUS_ONE:
            return value
    return QUESTION_MARK

def get_most_frequent_val(process):
    if not process.round_messages:
        return QUESTION_MARK, 0
    most_freq = max(process.round_messages, key=process.round_messages.get)
    count = process.round_messages[most_freq]
    if most_freq == QUESTION_MARK:
        process.round_messages.pop(most_freq)
        if process.round_messages:
            most_freq = max(process.round_messages, key=process.round_messages.get)
            count = process.round_messages[most_freq]
        else:
            return QUESTION_MARK, 0
    return most_freq, count

In [6]:
def agreement(process):
    try:
        current = process.input_val
        next_decide = False
        epoch = 0
        while True:
            epoch += 1
            print(f"[Process {process.id}] Entering epoch {epoch}")
            if process.faulty:
                fake_val = random.choice(["0", "1", "?", "X"])
                print(f"[Faulty Process {process.id}] Broadcasting in round 1 → {fake_val}")
                broadcast(process.id, epoch, 1, fake_val)

            else:
                broadcast(process.id, epoch, 1, current)

            if not next_decide:
                receive(process, epoch, 1)
                current = get_majority_value(process)
            process.round_messages.clear()

            if process.faulty:
                broadcast(process.id, epoch, 2, random.choice(["0", "1", "?", "X"]))
            else:
                broadcast(process.id, epoch, 2, current)

            if not next_decide:
                receive(process, epoch, 2)
                answer, number = get_most_frequent_val(process)
            process.round_messages.clear()

            broadcast(process.id, epoch, 3, WAITING_MESSAGE)

            if not next_decide:
                receive(process, epoch, 3, WAITING_MESSAGE)
            process.round_messages.clear()

            try:
                coin = QCF.quantum_coin_flip(processes, process, epoch)

            except Exception as e:
                print(f"[Error in coin flip for Process {process.id}] {e}")
                coin = random.randint(0, 1)

            if next_decide:
                break

            if number >= HALF_PLUS_ONE:
                current = answer
                next_decide = True
                process.decision_epoch = epoch
                decision_lock.acquire()
                global first_to_decide
                if first_to_decide is None:
                    first_to_decide = process.id
                decision_lock.release()
            elif number >= 1:
                current = answer
            else:
                current = coin

        process.output = current
        return current
    
    except Exception as e:
        print(f"[Process {process.id}] crashed unexpectedly: {e}")
        # Optional: set output so it’s visible in results
        process.output = "CRASHED"
        return

In [7]:
inputs = [random.randint(0, 1) for _ in range(n)]
faulty_ids = set(random.sample(range(n), t))

processes = [Process(id=i, input_val=inputs[i], faulty=(i in faulty_ids)) for i in range(n)]

threads = []
for pr in processes:
    thr = Thread(target=agreement, args=(pr,))
    thr.start()
    threads.append(thr)

for thr in threads:
    thr.join(timeout=20)

print("\n--- Final Decisions ---")
for pr in processes:
    print(f"Process {pr.id} | Output: {pr.output} | Faulty: {pr.faulty} | Decision Epoch: {pr.decision_epoch}")

print(f"\nFirst to decide: Process {first_to_decide}")


[Process 0] Entering epoch 1
[Process 0] BROADCAST round 1, epoch 1 → 1
[Process 0] WAITING to receive round 1 @ epoch 1
num_received_messages: 0, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive:  3
num_received_messages: 1, round: 1
actual_alive

In [8]:
test_all(processes, first_to_decide, broadcasted_messages)

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'